In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
import numpy as np
from datetime import datetime


def getRolledSeries(pathIn, key='bars/ES_10K'):
	series=pd.read_hdf(pathIn, key=key)
    series['Time'] = pd.to_datetime(series['Time'].format('%Y-%m-%d %H:%M:%S%f'))
    series=series.set_index('Time')
    gaps=rollGaps(series)
    for fld in ['Close', 'VWAP']:series[fld]-=gaps
    return series

def rollGaps(series,dictio={'Instrument':'FUT_CUR_GEN_TICKERS','Open':'PX_OPEN','Close':'PX_LAST'},matchEnd=True):
    #compite gaps at each roll, between previous close and next open
    rollDates=series[dictio['Instrument']].drop_duplicates(keep='first').index
    gaps=series[dictio['Close']]*0
    iloc=list(series.index)
    iloc=[iloc.index(i)-1 for i in rollDates])
    gaps.loc[rollDates[1:]]=series[dictio['Open']].loc[rollDates[1:]]-series[dictio['Close']].iloc[iloc[:-1]].values
    gaps=gaps.cumsum()
    if matchEnd:
        gaps-=gaps.iloc[-1]
    return gaps
